Step 3 : The Heuristic Separator
 
 Instead of training a Neural Network, we explicitly code the paper's conclusion as a scoring rule.
 **The Formula:** `Score = 0.7 * Efficacy + 0.3 * (1 - Parallelism)`
 **Logic:**
 1. Generate candidates.
 2. Calculate features.
 3. Score them using the formula.
 4. Sort and pick the Top-K.

In [ ]:
import networkx as nx
import numpy as np
import random
from pyscipopt import Model, Sepa, SCIP_RESULT, SCIP_PARAMSETTING

# 1. Feature Extractor (same as before)
def get_features(model, row):
    cols = row.getCols()
    vals = row.getVals()
    cut_coeffs = np.array(vals)
    if len(cut_coeffs) == 0: return np.zeros(13, dtype=np.float32)

    obj_coeffs = []
    for col in cols:
        try:
            var = col.getVar()
            c = var.getObj()
        except:
            c = col.getObj()
        obj_coeffs.append(c)
    obj_coeffs = np.array(obj_coeffs)
    
    # 1: Parallelism
    norm_cut = np.linalg.norm(cut_coeffs)
    norm_obj = np.linalg.norm(obj_coeffs)
    f_parallelism = 0.0
    if norm_cut > 1e-9 and norm_obj > 1e-9:
        f_parallelism = abs(np.dot(cut_coeffs, obj_coeffs) / (norm_cut * norm_obj))
        
    # 2: Efficacy
    try:
        f_efficacy = model.getCutEfficacy(row)
    except:
        activity = model.getRowActivity(row)
        rhs = row.getRhs()
        f_efficacy = max(0, activity - rhs) / norm_cut if norm_cut > 1e-9 else 0.0
    
    # We only need these two features to run the fomular, but we keep the vector in 13 dimentions in case.
    features = np.zeros(13)
    features[8] = f_parallelism
    features[9] = f_efficacy
    return features

# 2. Heuristic Separator
class HeuristicSepa(Sepa):
    def __init__(self):
        self.stats = {"cuts_added": 0, "total_score": 0.0}

    def sepaexeclp(self):
        model = self.model
        
        # A.Generate Candidates
        vars = model.getVars()
        fractional_vars = [v for v in vars if 0.001 < model.getSolVal(None, v) < 0.999]
        # close to 0.5
        fractional_vars.sort(key=lambda v: abs(model.getSolVal(None, v) - 0.5))
        
        candidates = []
        # 50 candidates
        for i, v in enumerate(fractional_vars[:50]):
            val = model.getSolVal(None, v)
            name = f"h_cut_{i}"
            if val < 0.5:
                row = model.createEmptyRowSepa(self, name, lhs=None, rhs=0.0)
            else:
                row = model.createEmptyRowSepa(self, name, lhs=1.0, rhs=None)
            model.addVarToRow(row, v, 1.0)
            
            if model.isCutEfficacious(row):
                candidates.append(row)
        
        if not candidates:
            return {"result": SCIP_RESULT.DIDNOTFIND}

        # B.Score
        scored_candidates = []
        for row in candidates:
            feats = get_features(model, row)
            
            # [fomular from the paper]
            # Efficacy (Feat 9): the bigger, the cut is deeper
            # Parallelism (Feat 8): the smaller the better(1 - Parallelism)
            
            efficacy = feats[9]
            parallelism = feats[8]
            
            # Score = 0.8 * Efficacy + 0.2 * Orthogonality
            score = (0.8 * efficacy) + (0.2 * (1.0 - parallelism))
            
            scored_candidates.append((score, row))
        
        # C.Order & Quantity)
        # 1. Order: from high to low
        scored_candidates.sort(key=lambda x: x[0], reverse=True)
        
        # 2. Quantity:
        # Top 10 used here
        top_k = scored_candidates[:10]
        
        # D. Execute
        for score, row in top_k:
            # forcecut=True We don't want SCIP fliter the cuts
            try:
                model.addCut(row, forcecut=True)
            except:
                model.addCut(row)
            
            self.stats["cuts_added"] += 1
            self.stats["total_score"] += score

        return {"result": SCIP_RESULT.SEPARATED}

# 3. Comparison
def test_heuristic():
    print(f"Testing Heuristic Rule...")
    
    # generate a MIS problem
    NODES = 100
    g = nx.gnp_random_graph(NODES, 0.15, seed=2505153)
    
    model = Model("Heuristic_Test")
    model.hideOutput()
    x = {n: model.addVar(vtype="B") for n in g.nodes()}
    for u, v in g.edges():
        model.addCons(x[u] + x[v] <= 1)
    model.setObjective(sum(x.values()), "maximize")
    
    # Disable SCIP default preprocessing and force it to use our cut
    model.setPresolve(SCIP_PARAMSETTING.OFF)
    model.setIntParam("separating/maxrounds", 10)
    
    # use our seperator
    my_sepa = HeuristicSepa()
    model.includeSepa(my_sepa, "HeuristicSepa", "", priority=100000, freq=1)
    
    model.optimize()
 
    print(f"   Optimal Value: {model.getObjVal()}")
    print(f"   Total Cuts Added by Rule: {my_sepa.stats['cuts_added']}")
    print(f"   Avg Score of Added Cuts: {my_sepa.stats['total_score'] / max(1, my_sepa.stats['cuts_added']):.4f}")

test_heuristic()

Testing Heuristic Rule...
   Optimal Value: 20.0
   Total Cuts Added by Rule: 10
   Avg Score of Added Cuts: 0.4000


In this section, we will conduct a more detailed comparative experiment to see more SCIP experiments with default parameters and learned policy. The basic logic is the same as the last part.

In [5]:
import networkx as nx
import numpy as np
import time
from pyscipopt import Model, Sepa, SCIP_RESULT, SCIP_PARAMSETTING

# 1.Heuristic Separator
# policy: Score = 0.8 * Efficacy + 0.2 * (1 - Parallelism)

class HeuristicSepa(Sepa):
    def __init__(self):
        self.cuts_added = 0

    def sepaexeclp(self):
        model = self.model
        
        # 1. Find Devider Variables
        vars = model.getVars()
        fractional_vars = [v for v in vars if 0.001 < model.getSolVal(None, v) < 0.999]
        if not fractional_vars: return {"result": SCIP_RESULT.DIDNOTFIND}
        
        # 2. Generate Canditates
        fractional_vars.sort(key=lambda v: abs(model.getSolVal(None, v) - 0.5))
        candidates = []
        for i, v in enumerate(fractional_vars[:50]):
            val = model.getSolVal(None, v)
            name = f"h_cut_{i}"
            if val < 0.5:
                row = model.createEmptyRowSepa(self, name, lhs=None, rhs=0.0)
            else:
                row = model.createEmptyRowSepa(self, name, lhs=1.0, rhs=None)
            model.addVarToRow(row, v, 1.0)
            
            if model.isCutEfficacious(row):
                candidates.append(row)
        
        if not candidates: return {"result": SCIP_RESULT.DIDNOTFIND}

        # 3. Score(The same as before)
        scored_cuts = []
        for row in candidates:
            cols = row.getCols()
            vals = row.getVals()
            alpha = np.array(vals)
            
            c_vec = []
            for col in cols:
                c_val = col.getVar().getObj()
                c_vec.append(c_val)
            c_vec = np.array(c_vec)
            
            # Parallelism
            # dot_product / (norm_alpha * norm_c)
            norm_alpha = np.linalg.norm(alpha)
            norm_c = np.linalg.norm(c_vec)
            
            parallelism = 0.0
            if norm_alpha > 1e-9 and norm_c > 1e-9:
                parallelism = abs(np.dot(alpha, c_vec)) / (norm_alpha * norm_c)
            
            # Efficacy
            # Efficacy = Distance from LP solution to the cut
            
            efficacy = model.getCutEfficacy(row)


            score = 0.8 * efficacy + 0.2 * (1.0 - parallelism)
            
        scored_cuts.append((score, row))
        
        # 4. rank Top 10
        scored_cuts.sort(key=lambda x: x[0], reverse=True)
        top_k = scored_cuts[:10]
        
        # 5. add Cuts
        for score, row in top_k:
            try:
                model.addCut(row, forcecut=True)
                self.cuts_added += 1
            except:
                pass

        return {"result": SCIP_RESULT.SEPARATED}


def solve_mis_instance(seed, use_heuristic=False):
    #1. Generate MIS
    NODES = 150
    g = nx.gnp_random_graph(NODES, 0.15, seed=2505153)
    
    model = Model(f"MIS_{seed}")
    model.hideOutput()
    
    # model
    x = {n: model.addVar(vtype="B") for n in g.nodes()}
    for u, v in g.edges():
        model.addCons(x[u] + x[v] <= 1)
    model.setObjective(sum(x.values()), "maximize")
    
    # 2.Set Environment 
    # Shut down Presolve
    model.setPresolve(SCIP_PARAMSETTING.OFF)
    
    # Set Cut Parameter
    model.setIntParam("separating/maxrounds", 10)
    
    custom_sepa = None
    if use_heuristic:
        custom_sepa = HeuristicSepa()
        # priority=100000
        model.includeSepa(custom_sepa, "HeuristicSepa", "", priority=100000, freq=1)
    
    #3. Timer & slove
    start_time = time.time()
    model.optimize()
    end_time = time.time()
    
    # 4. results
    status = model.getStatus()
    obj_val = model.getObjVal()
    nodes = model.getNNodes()
    time_taken = end_time - start_time
    
    # Dual Bound
    # Logic: The cut is better，if the root nodes bound is tighter.（As for maximizztion，Bound is getting lower.）
    # Final Dual Bound is our metric
    dual_bound = model.getDualbound()
    
    added_count = custom_sepa.cuts_added if custom_sepa else 0
    
    return {
        "status": status,
        "nodes": nodes,
        "time": time_taken,
        "dual_bound": dual_bound,
        "custom_cuts": added_count
    }

def run_comparison():
    SEED = 42
    print(f"Graph Nodes: 150")
    
    
    # Round 1: Default SCIP
    print("running default")
    res_default = solve_mis_instance(SEED, use_heuristic=False)
    
    # Round 2: Heuristic SCIP
    print("running heuristic")
    res_heuristic = solve_mis_instance(SEED, use_heuristic=True)
    
    #result
    print("\nEfficiency Comparison:")
    print(f"{'Metric':<20} | {'Default SCIP':<15} | {'With Heuristic':<15} | {'Improvement'}")
    
    # Nodes (metrics)
    d_nodes = res_default['nodes']
    h_nodes = res_heuristic['nodes']
    imp_nodes = f"{(d_nodes - h_nodes)/d_nodes * 100:.1f}%" if d_nodes > 0 else "N/A"
    print(f"{'Total Nodes':<20} | {d_nodes:<15} | {h_nodes:<15} | {imp_nodes} (Less is Better)")
    
    # Time
    d_time = res_default['time']
    h_time = res_heuristic['time']
    print(f"{'Time (seconds)':<20} | {d_time:<15.4f} | {h_time:<15.4f} | Diff: {h_time - d_time:.4f}s")
    
    d_bound = res_default['dual_bound']
    h_bound = res_heuristic['dual_bound']
    print(f"{'Final Dual Bound':<20} | {d_bound:<15.4f} | {h_bound:<15.4f} | (Higher is Better)")
    
 
    print(f"We add {res_heuristic['custom_cuts']} cuts based on learned policy")
    
    if h_nodes < d_nodes:
        print("Conclusion: The paper successfully reduced the number of search nodes by cutting the plane according to the learned policy! Proved effectiveness.")


if __name__ == "__main__":
    run_comparison()

Graph Nodes: 150
running default
running heuristic

Efficiency Comparison:
Metric               | Default SCIP    | With Heuristic  | Improvement
Total Nodes          | 92837           | 1               | 100.0% (Less is Better)
Time (seconds)       | 284.4590        | 0.2247          | Diff: -284.2343s
Final Dual Bound     | 27.0000         | 24.0000         | (Higher is Better)
We add 13 cuts based on learned policy
Conclusion: The paper successfully reduced the number of search nodes by cutting the plane according to the learned policy! Proved effectiveness.
